In [ ]:
!pip intall numpy
!pip intall pandas
!pip intall matplotlib
!pip intall seaborn
!pip intall warnings
!pip intall tqdm
!pip intall ydata_profiling


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
import os
from tqdm import tqdm
from datetime import date, timedelta
from datetime import datetime as dt
import os


In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
def wape(y_true: np.array, y_pred: np.array):
    return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))


In [ ]:
current_directory = os.getcwd()

filename = "pr_df.csv"

file_path = os.path.join(current_directory, filename)
pr_df = pd.read_csv(file_path)

In [ ]:
pr_df.head(3)

In [ ]:
pr_df

In [ ]:
pr_df_hist_list = list(pr_df.columns)

In [ ]:
pr_df_hist_list.pop(0)

In [ ]:
filename = "sales_df_train.csv"

file_path = os.path.join(current_directory, filename)
sales_df_train = pd.read_csv('file_path')

In [ ]:
def grab_float_columns(df):
    return list(df.select_dtypes(include='float').columns)

In [ ]:
float_columns = grab_float_columns(sales_df_train)

In [ ]:
float_columns

In [ ]:
def check_nonzero_decimal(row):
    decimal_part = row - int(row)
    if decimal_part != 0:
        raise StopIteration  # Выбрасываем исключение для завершения apply



In [ ]:
lst=[]

In [ ]:
for i in float_columns:
    try:
        sales_df_train[i].apply(check_nonzero_decimal)
        lst.append(i)
    except StopIteration:
        print(F'В колонке {i} есть как минимум одно число, у которого после запятой есть цифра отличная от нуля! ') 

**Вывод:**

* Можем для экономии памяти перевести из `float` типа в `int`

In [ ]:
for i in float_columns:
    sales_df_train[i] = sales_df_train[i].astype('int')

In [ ]:
# столбец 'date' в формат datetime
sales_df_train['date'] = pd.to_datetime(sales_df_train['date'])

# столбец 'month' с номерами месяцев
sales_df_train['month'] = sales_df_train['date'].dt.month

# столбец 'day_of_week' с номерами дней недели (понедельник - 0, воскресенье - 6)
sales_df_train['day_of_week'] = sales_df_train['date'].dt.dayofweek

In [ ]:

#  столбец 'season' с указанием времени года
seasons = {
    1: 'Зима',
    2: 'Зима',
    3: 'Весна',
    4: 'Весна',
    5: 'Весна',
    6: 'Лето',
    7: 'Лето',
    8: 'Лето',
    9: 'Осень',
    10: 'Осень',
    11: 'Осень',
    12: 'Зима'
}
sales_df_train['season'] = sales_df_train['month'].map(seasons)

In [ ]:
filename = "st_df.csv"

file_path = os.path.join(current_directory, filename)
pr_st = pd.read_csv(filename)

In [ ]:
pr_st_active_only = pr_st.copy()

In [ ]:
pr_st_active_only = pr_st_active_only[pr_st_active_only['st_is_active']==1].reset_index(drop=True)

# Объединение таблиц 

In [ ]:
merged_df = sales_df_train.merge(pr_df, on='pr_sku_id', how='left')


In [ ]:
final_merged = merged_df.merge(pr_st_active_only, on='st_id', how='left')


In [ ]:
final_merged.isna().sum()

In [ ]:
final_merged.shape

729 - это данные для неактивных магазинов. Можем удалять смело.

In [ ]:
final_merged.dropna(inplace=True,ignore_index=True)

In [ ]:
final_merged.shape

In [ ]:
final_merged.isna().sum()

# Добавление признаков

In [ ]:
holiday_dict = {1:[i for i in range(1,10)],
2:[23],
3:[8],
5:[1,9],
6:[12],
11:[4],
12:[31]}

In [ ]:
def set_flag_holiday(row, holiday_dict):
    '''
    Функция размечает даты - праздник / не праздник
    :param row: Таблица
    :param holiday_dct: Словарь месяц:[число(1),...,число(i)]
    '''
    
    month = row['month']
    day = row['date'].day
    if month in holiday_dict and day in holiday_dict[month]:
        return 1
    return 0



In [ ]:
final_merged['holiday'] = final_merged.apply(lambda row: set_flag_holiday(row, holiday_dict), axis=1)


In [ ]:
final_merged = final_merged.sort_values(by='date')

In [ ]:
tmp_date = final_merged.copy()

In [ ]:
tmp_date['year'] = tmp_date['date'].dt.year
tmp_date['day'] =tmp_date['date'].dt.day

In [ ]:
# Предположим, что data_new - это ваш DataFrame
tmp_date.rename(columns={'date': 'date_x'}, inplace=True)

In [ ]:
# Создаем словарь для замены значений
season_mapping = {'Зима': 0, 'Весна': 1, 'Лето': 2, 'Осень': 3}

# Заменяем значения в столбце 'season' согласно словарю
tmp_date['season'] = tmp_date['season'].replace(season_mapping)

In [ ]:
data_group = tmp_date.groupby(['date_x', 'st_id', 'pr_sku_id']).agg({
                                                                 'day':'mean' , 
                                                                 'day_of_week':'mean', 
                                                                    'month':'mean',
                                                                    'year':'mean',
                                                                    'season':'mean',
                                                                 'holiday':'mean', 
                                                                 'pr_sales_in_units':'sum'
                                                                })

In [ ]:
tmp=data_group.copy()

In [ ]:
tmp = tmp.reset_index()

In [ ]:
unique_rec= tmp.drop_duplicates(subset=['date_x'])



In [ ]:
unique_rec.reset_index(drop=False,inplace=True)

In [ ]:
def after_holidays_n_days(row, n_before):
    if row['holiday'] == 1:
        return 1
    for i in range(1, n_before + 1):
        if row.name - i < 0:
            break
        if unique_rec.at[row.name - i, 'holiday'] == 1:
            return 1
    return 0

def before_holidays_n_days(row, n_after):
    if row['holiday'] == 1:
        return 1
    for i in range(1, n_after + 1):
        if row.name + i >= len(unique_rec):
            break
        if unique_rec.at[row.name + i, 'holiday'] == 1:
            return 1
    return 0


In [ ]:
n_before = 14  # Количество дней до и после праздника
unique_rec['before_holidays_n_days'] = unique_rec.apply(lambda row: before_holidays_n_days(row, n_before), axis=1)


In [ ]:
n_after= 7
unique_rec['after_holidays_n_days'] = unique_rec.apply(lambda row: after_holidays_n_days(row, n_after), axis=1)


In [ ]:
before_after_day = unique_rec.loc[:,['date_x','before_holidays_n_days','after_holidays_n_days']]

In [ ]:
tmp = tmp.merge(before_after_day, on='date_x', how='left')


In [ ]:
tmp= tmp.drop(columns=['day','day_of_week','month','season'],axis=1)

In [ ]:
tmp.head(2)

In [ ]:
# Предположим, что data_new - это ваш DataFrame
tmp.rename(columns={'date': 'date_x'}, inplace=True)

In [ ]:


tmp = tmp.reset_index().set_index('date_x')


In [ ]:
tmp.drop('index',axis=1,inplace=True)

In [ ]:
shop_list = tmp['st_id'].unique()
product_list = tmp['pr_sku_id'].unique()

In [ ]:
wape_results = {}
target = 'pr_sales_in_units'

In [ ]:
data_new = pd.DataFrame()
for shop in shop_list:
    data_chunk = tmp.query('st_id == \'{0}\''.format(shop))
    for product in product_list:
        data_chunk_2 = data_chunk.query('pr_sku_id == \'{0}\''.format(product))
        if data_chunk_2.shape[0] != 0:
            
            #data_chunk_2 = data_chunk_2
            data_chunk_2['month'] = data_chunk_2.index.month
            data_chunk_2['year'] = data_chunk_2.index.year

            idx = pd.date_range('{0}'.format(data_chunk_2.index.min()).replace('00:00:00', ''), '{0}'.format(data_chunk_2.index.max()).replace('00:00:00', ''))
        
            s = pd.Series({})

            s.index = pd.DatetimeIndex(s.index)
            s = s.reindex(idx, fill_value=0)
            s = pd.DataFrame(s, index=s.index)

            data_tmp = s.merge(data_chunk_2, left_index=True, right_on='date_x', how='left')
            data_tmp['st_id'] = data_tmp['st_id'].fillna(shop) 
            data_tmp['pr_sku_id'] = data_tmp['pr_sku_id'].fillna(product) 
            data_tmp['target_rolling_mean_3'] = data_tmp['pr_sales_in_units'].shift().rolling(3).mean()
           

            data_new = pd.concat([data_new, data_tmp])


data_new

In [ ]:
data_new.loc[data_new['pr_sales_in_units'].isna(), 'pr_sales_in_units'] = data_new[data_new['pr_sales_in_units'].isna()]['target_rolling_mean_3']

In [ ]:
data_new = data_new[~data_new['pr_sales_in_units'].isna()]

In [ ]:
data_new.head(10)

In [ ]:
data_new.loc[data_new['pr_sales_in_units'].isna(), 'pr_sales_in_units'] = data_new[data_new['pr_sales_in_units'].isna()]['target_rolling_mean_3']

In [ ]:
data_new = data_new[~data_new['pr_sales_in_units'].isna()]

In [ ]:
data_new = data_new.reset_index()
data_new['date'] = data_new['index']

In [ ]:
data_new.loc[data_new['date'].isna(), 'date'] = data_new[data_new['date'].isna()]['date_x']

In [ ]:
data_new = data_new.set_index('date')

In [ ]:
data_new = data_new.drop([0, 'index', 'date_x', 'target_rolling_mean_3'], axis=1)

In [ ]:
data_new.isna().sum()

In [ ]:
data_new= data_new.drop(columns=['month','year','holiday','before_holidays_n_days','after_holidays_n_days'],axis=1)

In [ ]:
data_new = data_new[data_new['pr_sales_in_units']>0]

In [ ]:
data_new = data_new.reset_index()

In [ ]:
data_new['month'] = data_new.date.dt.month
data_new['day_of_week'] = data_new.date.dt.dayofweek
data_new['day'] = data_new.date.dt.day

In [ ]:
data_new= data_new.drop(['month',"day_of_week","day"],axis=1)

In [ ]:
def make_features(data,cos_sin_columns = ['month','day_of_week','day'],    holiday_dict = {1:[i for i in range(1,10)],
    2:[23],
    3:[8],
    5:[1,9],
    6:[12],
    11:[4],
    12:[31]}):
    data['date'] = pd.to_datetime(data['date'])
    data['month'] = data['date'].dt.month
    
    #  столбец 'season' с указанием времени года
    seasons = {
        1: 0,
        2: 0,
        3: 1,
        4: 1,
        5: 1,
        6: 2,
        7: 2,
        8: 2,
        9: 3,
        10: 3,
        11: 3,
        12: 0
    }
    data['season'] = data['month'].map(seasons)
    data['day'] = data['date'].dt.day
    data['day_of_week'] =data['date'].dt.dayofweek
    def set_flag_holiday(row, holiday_dict):
        '''
        Функция размечает даты - праздник / не праздник
        :param row: Таблица
        :param holiday_dct: Словарь месяц:[число(1),...,число(i)]
        '''
    
        month = row['month']
        day = row['date'].day
        weekday = row['day_of_week']
        if (month in holiday_dict and day in holiday_dict[month]):
            return 1
        return 0


    data['holiday'] = data.apply(lambda row: set_flag_holiday(row, holiday_dict), axis=1)
    def after_holidays_n_days(row, n_before=14):
        if row['holiday'] == 1:
            return 1
        for i in range(1, n_before + 1):
            if row.name - i < 0:
                break
            if unique_rec.at[row.name - i, 'holiday'] == 1:
                return 1
        return 0

    def before_holidays_n_days(row, n_after=7):
        if row['holiday'] == 1:
            return 1
        for i in range(1, n_after + 1):
            if row.name + i >= len(unique_rec):
                break
            if unique_rec.at[row.name + i, 'holiday'] == 1:
                return 1
        return 0
    
    data = data.sort_values(by='date')
    tmp=data.copy()
    unique_rec= tmp.drop_duplicates(subset=['date'])
    unique_rec.reset_index(drop=False,inplace=True)
    unique_rec['before_holidays_n_days'] = unique_rec.apply(lambda row: before_holidays_n_days(row), axis=1)
    unique_rec['after_holidays_n_days'] = unique_rec.apply(lambda row: after_holidays_n_days(row), axis=1)
    before_after_day = unique_rec.loc[:,['date','before_holidays_n_days','after_holidays_n_days']]
    data = data.merge(before_after_day, on='date', how='left')
    
    
    def cos_sin_categorise(df,cos_column_name,sin_column_name,column_to_categorise):
        """
        Применяет cos-sin тригонометрическую категоризацию

        :param data: DataFrame содержащий данные
        :param cos_column_name: Название будущей колонки с cos
        :param sin_column_name: Название будущей колонки с sin
        :param column_to_categorise: Назване колонки, которое хотим категоризовать
        """
        df[cos_column_name] = np.cos((2*np.pi *df[column_to_categorise])/df[column_to_categorise].nunique())
        df[sin_column_name] = np.sin((2*np.pi *df[column_to_categorise])/df[column_to_categorise].nunique())
    
    for i in cos_sin_columns:
        cos_sin_categorise(data,f'cos_{i}',f'sin_{i}',i)

    def set_flag_holiday_weekend(row, holiday_dict):
        '''
        Функция размечает даты - праздник / не праздник
        :param row: Таблица
        :param holiday_dct: Словарь месяц:[число(1),...,число(i)]
        '''

        month = row['month']
        day = row['date'].day
        weekday = row['day_of_week']
        if (month in holiday_dict and day in holiday_dict[month])or (weekday in [5,6]):
            return 1
        return 0
    data['holiday'] = data.apply(lambda row: set_flag_holiday_weekend(row, holiday_dict), axis=1)
    data['date'] = pd.to_datetime(data['date'])
    sales_df_train['date'] = pd.to_datetime(sales_df_train['date'])
    data.drop(['day','month','day_of_week'],axis=1,inplace=True)
    data = data.merge(pr_df, on='pr_sku_id', how='left')

    data = data.merge(pr_st_active_only, on='st_id', how='left')
    data = data.merge(sales_df_train.loc[:,['date','pr_sku_id','pr_sales_type_id']],on=['date','pr_sku_id'],how='left')
    features_columns_list = ['st_id', 'pr_sku_id', 'date', 'holiday', 'season', 'cos_month', 'sin_month',
       'cos_day_of_week', 'sin_day_of_week', 'cos_day', 'sin_day',
       'before_holidays_n_days', 'after_holidays_n_days','pr_sales_type_id',
       'pr_uom_id']
    
    data = data.reindex(columns=features_columns_list)

    return data

In [ ]:
data_new = make_features(data_new)

In [ ]:
days_to_keep_in_test=14

In [ ]:
np.random.seed(42)
for season in tqdm(season_dataframes.keys(), desc="Processing Season"):
    test_end_date = season_dataframes[season]['date'].max()
    # Вычисляем начальную дату, отсчитывая от конца
    test_start_date = test_end_date - timedelta(days=days_to_keep_in_test - 1)
    # Фильтруем данные для тестовой выборки
    data_new_test = season_dataframes[season][(season_dataframes[season]['date'] >= test_start_date) & (season_dataframes[season]['date'] <= test_end_date)]
    # Фильтруем данные для тренировочной выборки (оставляем все, кроме тестовой части)
    data_new_train = season_dataframes[season][season_dataframes[season]['date'] < test_start_date]    
    lag_df_train = pd.DataFrame()
    # Перебираем уникальные пары магазин-товар
    for store in tqdm(data_new_train.st_id.unique(), desc="Processing Stores Train"):
        for product in data_new_train.pr_sku_id.unique():
            subset = data_new_train[(data_new_train['st_id'] == store) & (data_new_train['pr_sku_id'] == product)].copy()
            # Фильтруем DataFrame для текущей пары магазин-товар
            subset['lag_1_sales'] = subset['pr_sales_in_units'].shift(periods=1)
            subset['lag_2_sales'] = subset['pr_sales_in_units'].shift(periods=2)
            subset['lag_7_sales'] = subset['pr_sales_in_units'].shift(periods=7)
            subset['lag_14_sales'] = subset['pr_sales_in_units'].shift(periods=14)
            # Вычисляем скользящее среднее за 7 дней
            subset['rolling_mean_2'] = subset['pr_sales_in_units'].shift().rolling(window=2).mean()
            subset['rolling_mean_7'] = subset['pr_sales_in_units'].shift().rolling(window=7).mean()
            subset['rolling_mean_14'] = subset['pr_sales_in_units'].shift().rolling(window=14).mean()


            # Добавляем результаты в общий DataFrame
            lag_df_train = pd.concat([lag_df_train, subset])
            lag_df_train = lag_df_train.fillna(value=0)
    # Создаем новый DataFrame для хранения лагов
    lag_df_test = pd.DataFrame()

    # Перебираем уникальные пары магазин-товар
# Перебираем уникальные пары магазин-товар
    for store in tqdm(data_new_test.st_id.unique(), desc="Processing Stores Test"):
        for product in data_new_test.pr_sku_id.unique():
            # Фильтруем DataFrame для текущей пары магазин-товар
            subset = data_new_test[(data_new_test['st_id'] == store) & (data_new_test['pr_sku_id'] == product)].copy()
            # Создаем лаги
            subset['lag_1_sales'] = subset['pr_sales_in_units'].shift(periods=1)
            subset['lag_2_sales'] = subset['pr_sales_in_units'].shift(periods=2)
            subset['lag_7_sales'] = subset['pr_sales_in_units'].shift(periods=7)
            subset['lag_14_sales'] = subset['pr_sales_in_units'].shift(periods=14)
            # Вычисляем скользящее среднее за 7 дней
            subset['rolling_mean_2'] = subset['pr_sales_in_units'].shift().rolling(window=2, min_periods=2).mean()
            subset['rolling_mean_7'] = subset['pr_sales_in_units'].shift().rolling(window=7, min_periods=7).mean()
            subset['rolling_mean_14'] = subset['pr_sales_in_units'].shift().rolling(window=14, min_periods=14).mean()
            # Заполняем пропущенные значения скользящим средним
            # Добавляем результаты в общий DataFrame
            lag_df_test = pd.concat([lag_df_test, subset])
            lag_df_test = lag_df_test.fillna(value=0)



In [ ]:
def calculate_rolling_outliers(group):
    group = group.sort_values(by='date')

    group['rolling_mean'] = group['pr_sales_in_units'].expanding().mean()
    group['rolling_std'] = group['pr_sales_in_units'].expanding().std()
    group['zscore'] = (group['pr_sales_in_units'] - group['rolling_mean']) / group['rolling_std']

    group['q1'] = group['pr_sales_in_units'].expanding().quantile(0.25)
    group['q3'] = group['pr_sales_in_units'].expanding().quantile(0.75)
    group['iqr'] = group['q3'] - group['q1']
    group['lower_bound'] = group['q1'] - 1.5 * group['iqr']
    group['upper_bound'] = group['q3'] + 1.5 * group['iqr']
    group['is_outlier'] = ((group['zscore'].abs() > 3) |
                           (group['pr_sales_in_units'] < group['lower_bound']) |
                           (group['pr_sales_in_units'] > group['upper_bound'])).astype(int)

    return group

In [ ]:

def detect_outliers(df_train, df_test):
    df_train = df_train.groupby(['pr_sku_id', 'st_id'], group_keys=False).apply(calculate_rolling_outliers).reset_index(drop=True)

    outlier_columns = ['q1', 'q3', 'iqr', 'lower_bound', 'upper_bound']
    last_thresholds = df_train.groupby(['pr_sku_id', 'st_id'])[outlier_columns].last().reset_index()
    df_test = pd.merge(df_test, last_thresholds, on=['pr_sku_id', 'st_id'], how='left')
    df_test['is_outlier'] = ((df_test['pr_sales_in_units'] < df_test['lower_bound']) |
                             (df_test['pr_sales_in_units'] > df_test['upper_bound'])).astype(int)
    df_test.drop(outlier_columns, axis=1, inplace=True)

    drop_cols_from_train = ['rolling_mean', 'rolling_std', 'zscore'] + outlier_columns
    df_train.drop(drop_cols_from_train, axis=1, inplace=True)

    return df_train, df_test

In [ ]:
lag_df_train,lag_df_test = detect_outliers(lag_df_train,lag_df_test)

In [ ]:
lag_df_train.to_csv('train.csv')
lag_df_test.to_csv('test.csv')